# Import modules

In [1]:
import ROOT
%jsroot on
ROOT.gROOT.ProcessLine("gErrorIgnoreLevel = 2001;")
ROOT.gROOT.LoadMacro('RooCMSShape.cc+')
ROOT.gROOT.LoadMacro('RooErfExp.cc+')

Welcome to JupyROOT 6.26/02


0

# Read root file

In [20]:
year = 2018
f = ROOT.TFile("DYJetsToLL_M-50_TuneCP5_13TeV-madgraphMLM-pythia8/DY{}.root".format(year), "read")
hNoCorr = f.Get("demo/hZNoRoccor")
hCorr = f.Get("demo/hZRoccor")
noRoccor = ROOT.RooDataHist("noRoccor", "noRoccor", ROOT.RooArgList(x), ROOT.RooFit.Import(hNoCorr))

# Fitting PDF

In [38]:
x = ROOT.RooRealVar("x", "m_{Z}", 75, 105)
mass = ROOT.RooRealVar("mass", "mass of Z boson", 0, -5, 5)
sigma = ROOT.RooRealVar("sigma", "width of Gauss", 0.9, 0.05, 5.0)

histPdf = ROOT.RooHistPdf("histPdf", "histPdf", ROOT.RooArgSet(x), noRoccor)

fracSig = ROOT.RooRealVar("fracSig", "fracSig", 0.9, 0, 1)

gauss = ROOT.RooGaussian("gauss", "gauss", x, mass, sigma)
x.setBins(100000)
sig = ROOT.RooFFTConvPdf("sig", "sig", x, histPdf, gauss)

acms = ROOT.RooRealVar("acms", "acms", 70., 50., 90.)
beta = ROOT.RooRealVar("beta", "beta", 0.05, 0.01, 0.08)
gamma = ROOT.RooRealVar("gamma", "gamma", 0.1, -2, 2)
peak = ROOT.RooRealVar("peak", "peak", 91, 90, 92)

bkg = ROOT.RooCMSShape("bkg", "bkg", x, acms, beta, gamma, peak)

model = ROOT.RooAddPdf("model", "model", ROOT.RooArgList(sig, bkg), fracSig)
#gauss = ROOT.RooGaussian("gauss", "gauss", x, mass, sigma1)

[#0] ERROR:Caching -- The internal binning of variable x is not uniform. The numerical FFT will likely yield wrong results.


# Fit

In [39]:
#roccor = ROOT.RooDataHist("roccor", "roccor", ROOT.RooArgList(x), ROOT.RooFit.Import(hCorr))
print("Fitting No RoccoR")
model.fitTo(noRoccor, ROOT.RooFit.NumCPU(12, 0))
#print("Fitting RoccoR")
#fitCorrPdf.fitTo(roccor)

Fitting No RoccoR


[#1] INFO:Minimization -- RooAbsMinimizerFcn::setOptimizeConst: activating const optimization
[#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x600000fcb660 with pdf histPdf_CONV_gauss_CACHE_Obs[x]_NORM_x for nset (x) with code 0
[#1] INFO:NumericIntegration -- RooRealIntegral::init(bkg_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)
 **********
 **   19 **SET PRINT           1
 **********
 **********
 **   20 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 acms         7.00000e+01  4.00000e+00    5.00000e+01  9.00000e+01
     2 beta         5.00000e-02  7.00000e-03    1.00000e-02  8.00000e-02
     3 fracSig      9.00000e-01  5.00000e-02    0.00000e+00  1.00000e+00
     4 gamma        1.00000e-01  4.00000e-01   -2.00000e+00  2.00000e+00
     5 mass         0.00000e+00  1.00000e+00   -5.00000e+00  5.00000e+00
     6 peak         9.10000e+01  2.00000e-01    9.00000e+01  9.20000e+01

# Plot

In [40]:
xFrameNoCorr = x.frame()
#xFrameCorr = x.frame()
noRoccor.plotOn(xFrameNoCorr)
model.plotOn(xFrameNoCorr, ROOT.RooFit.LineColor(ROOT.kRed))
#roccor.plotOn(xFrameCorr, ROOT.RooFit.MarkerStyle(24))
#fitCorrPdf.plotOn(xFrameCorr, ROOT.RooFit.LineStyle(2))
c = ROOT.TCanvas()
c.Draw()
xFrameNoCorr.Draw()
#xFrameCorr.Draw("same")

[#1] INFO:Caching -- RooAbsCachedPdf::getCache(sig) creating new cache 0x60000cfa22b0 with pdf histPdf_CONV_gauss_CACHE_Obs[x]_NORM_x for nset (x) with code 0
[#1] INFO:NumericIntegration -- RooRealIntegral::init(bkg_Int[x]) using numeric integrator RooIntegrator1D to calculate Int(x)


In [41]:
print(xFrameNoCorr.chiSquare(7))

6261.131638186977


# Test auto fitting

In [44]:
x = ROOT.RooRealVar("x", "m_{Z}", 75, 105)
mass = ROOT.RooRealVar("mass", "mass of Z boson", mass.getVal(), 90, 92)
gamma = ROOT.RooRealVar("gamma", "width of BW", gamma.getVal(), 2, 4)
sigma = ROOT.RooRealVar("sigma", "width of Gauss", sigma.getVal(), 0, 1)
massCorr = ROOT.RooRealVar("massCorr", "mass of Z boson", massCorr.getVal(), 90, 92)
gammaCorr = ROOT.RooRealVar("gammaCorr", "width of BW", gammaCorr.getVal(), 2, 4)
sigmaCorr = ROOT.RooRealVar("sigmaCorr", "width of Gauss", sigmaCorr.getVal(), 0, 1)
fitNoCorrPdf = ROOT.RooVoigtian("fitPdf", "voigtian", x, mass, gamma, sigma)
fitCorrPdf = ROOT.RooVoigtian("fitPdf", "voigtian", x, massCorr, gammaCorr, sigmaCorr)